In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview/spark-3.0.0-preview-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from time import sleep
from pyspark.streaming import StreamingContext
#Spark Contexto
#df = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

from pyspark.sql import SparkSession

ss = SparkSession \
    .builder \
    .appName("Python Spark create Streaming example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
sc.version 

##AQUI O LINK DO GOOGLE DRIVE COM O CSV DE DADOS REAIS
https://drive.google.com/open?id=1iyRxudok8eBOqLRWet5og99-d1kw5xy_K93Chc_zU4w


In [0]:
transaction_detail_df = ss.read.csv("/content/transaction_detail.csv")

##AQUI MELHOR SERIA MUDAR O ARQUIVO PARA PARQUET, E MAIS RAPIDO, MAS COMO E SO UM TESTE VOU DEIXAR CSV

In [0]:
transaction_detail_df.printSchema()

In [0]:
transaction_detail_df.show(2, False)

In [0]:
transaction_detail_df = ss.read.option("header", True).option("inferSchema", True).csv("/content/transaction_detail.csv")

In [0]:
transaction_detail_df.printSchema()

In [0]:
transaction_detail_df.count()

In [0]:
transaction_detail_df.show(2, False)

In [0]:
transaction_detail_df_stg1 = transaction_detail_df.select("transaction_card_type", "transaction_country_name", "transaction_amount")

In [0]:
transaction_detail_df_stg1.show(5, False)

In [0]:
transaction_detail_df_stg1.groupby('transaction_card_type').agg({'transaction_amount': 'sum'}).show()

In [0]:
transaction_detail_df_stg1.groupby('transaction_country_name').agg({'transaction_amount': 'sum'}).show()

In [0]:
transaction_detail_df_stg1.groupby('transaction_country_name').count().show()

In [0]:
transaction_detail_df_stg1.filter(transaction_detail_df_stg1.transaction_country_name == 'Brasil').show(5, False)

In [0]:
transaction_detail_df_stg1.select("transaction_country_name").distinct().show()

In [0]:
transaction_detail_df_stg1.createTempView("transaction_detail_subset")

In [0]:
transaction_detail_count_by_country_df = ss.sql("SELECT transaction_country_name, COUNT(*) AS transaction_count FROM transaction_detail_subset GROUP BY transaction_country_name")
transaction_detail_count_by_country_df.show()

In [0]:
# import seaborn package
import seaborn as sns
sns.set()

# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
transaction_detail_count_by_country_pd_df = transaction_detail_count_by_country_df.toPandas()

In [0]:
pkmn_type_colors = ['#78C850',  # Grass
                    '#F08030',  # Fire
                    '#6890F0',  # Water
                    '#A8B820',  # Bug
                    '#A8A878',  # Normal
                    '#A040A0',  # Poison
                    '#F8D030',  # Electric
                    '#E0C068',  # Ground
                    '#EE99AC',  # Fairy
                    '#C03028',  # Fighting
                    '#F85888',  # Psychic
                    '#B8A038',  # Rock
                    '#705898',  # Ghost
                    '#98D8D8',  # Ice
                    '#7038F8',  # Dragon
                   ]

In [0]:
# Bar Plot
sns.barplot(x='transaction_country_name', y='transaction_count', data=transaction_detail_count_by_country_pd_df, palette=pkmn_type_colors)
 
# Rotate x-labels
plt.xticks(rotation=-65)

In [0]:
transaction_detail_count_by_country_pd_df.head(25)

In [0]:
transaction_detail_count_by_country_pd_df.head(25)

In [0]:
# Bar Plot - Horizontal
sns.barplot(x='transaction_count', y='transaction_country_name', data=transaction_detail_count_by_country_pd_df, palette=pkmn_type_colors)
 
# Rotate x-labels
plt.xticks(rotation=-65)